In [1]:
import numpy as np
import pandas as pd        # For loading and processing the dataset
import tensorflow as tf    # Of course, we need TensorFlow.
from sklearn.model_selection import train_test_split

# Read the CSV input file and show first 5 rows
df_train = pd.read_csv('input/train.csv')
df_train.head(5)

# To make 'Sex' numeric, we replace 'female' by 0 and 'male' by 1
df_train['Sex'] = df_train['Sex'].map({'female':0, 'male':1}).astype(int) 

# We replace 'Embarked' by three dummy variables 'Embarked_S', 'Embarked_C', and 'Embarked Q',
# which are 1 if the person embarked there, and 0 otherwise.
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)
df_train = df_train.drop('Embarked', axis=1)

# We normalize the age and the fare by subtracting their mean and dividing by the standard deviation
age_mean = df_train['Age'].mean()
age_std = df_train['Age'].std()
df_train['Age'] = (df_train['Age'] - age_mean) / age_std

fare_mean = df_train['Fare'].mean()
fare_std = df_train['Fare'].std()
df_train['Fare'] = (df_train['Fare'] - fare_mean) / fare_std

# In many cases, the 'Age' is missing - which can cause problems. Let's look how bad it is:
print("Number of missing 'Age' values: {:d}".format(df_train['Age'].isnull().sum()))

# A simple method to handle these missing values is to replace them by the mean age.
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

df_train = df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# With that, we're almost ready for training
df_train.head()

# Finally, we convert the Pandas dataframe to a NumPy array, and split it into a training and test set
X_train = df_train.drop('Survived', axis=1).to_numpy()
y_train = df_train['Survived'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

# We'll build a classifier with two classes: "survived" and "didn't survive",
# so we create the according labels
# This is taken from https://www.kaggle.com/klepacz/titanic/tensor-flow
labels_train = (np.arange(2) == y_train[:,None]).astype(np.float32)
labels_test = (np.arange(2) == y_test[:,None]).astype(np.float32)

Number of missing 'Age' values: 177


In [2]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,-0.530005,1,0,-0.502163,0,0,1
1,1,1,0,0.571430,1,0,0.786404,1,0,0
2,1,3,0,-0.254646,0,0,-0.488580,0,0,1
3,1,1,0,0.364911,1,0,0.420494,0,0,1
4,0,3,1,0.364911,0,0,-0.486064,0,0,1


In [3]:
pd.DataFrame(data=X_train)

,0,1,2,3,4,5,6,7,8
0,1.0,1.0,2.003921e-16,0.0,0.0,3.814891,0.0,0.0,1.0
1,3.0,1.0,-6.676845e-01,0.0,0.0,-0.456886,0.0,0.0,1.0
2,2.0,1.0,-8.053640e-01,0.0,0.0,-0.386454,0.0,0.0,1.0
3,3.0,1.0,-3.234859e-01,0.0,0.0,-0.506188,0.0,0.0,1.0
4,3.0,0.0,-1.631441e+00,4.0,2.0,-0.018699,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
707,3.0,0.0,8.955238e-02,1.0,0.0,-0.285837,0.0,0.0,1.0
708,3.0,1.0,2.003921e-16,0.0,0.0,-0.502666,1.0,0.0,0.0
709,2.0,1.0,-1.837960e+00,1.0,1.0,-0.124850,0.0,0.0,1.0
710,3.0,1.0,-5.300051e-01,0.0,0.0,-0.489167,0.0,0.0,1.0


In [4]:
model = tf.keras.Sequential([
    # tf.keras.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
  loss=tf.keras.losses.BinaryCrossentropy(
    from_logits=True, label_smoothing=0, reduction="auto", name="binary_crossentropy"),
  metrics=['accuracy'],
)

In [5]:
x = model.layers[4].get_weights()

In [6]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
model.fit(
  X_train,
  labels_train, # training targets
  epochs=50,
  callbacks=[tbCallBack]
)

Epoch 1/50
 1/23 [>.............................] - ETA: 0s - loss: 0.7361 - accuracy: 0.4375WARNING:tensorflow:From /opt/conda/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
23/23 [==============================] - 0s 2ms/step - loss: 0.7253 - accuracy: 0.4958
Epoch 2/50
23/23 [==============================] - 0s 1ms/step - loss: 0.7206 - accuracy: 0.5365
Epoch 3/50
23/23 [==============================] - 0s 1ms/step - loss: 0.7143 - accuracy: 0.5576
Epoch 4/50
23/23 [==============================] - 0s 1ms/step - loss: 0.7082 - accuracy: 0.5941
Epoch 5/50
23/23 [==============================] - 0s 1ms/step - loss: 0.7008 - accuracy: 0.6236
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 0.7006 - accuracy: 0.6011
Epoch 7/50
23/23 [==============================] - 

In [7]:
op = model.predict(X_test)
print(op)

[[0.7183902  0.2816098 ]
 [0.9592674  0.04073264]
 [0.46977243 0.53022754]
 [0.89555806 0.10444196]
 [0.960958   0.03904193]
 [0.90012753 0.09987248]
 [0.937597   0.062403  ]
 [0.93007326 0.06992667]
 [0.9164197  0.08358035]
 [0.9618704  0.03812966]
 [0.8336214  0.16637856]
 [0.89555806 0.10444196]
 [0.5638575  0.43614256]
 [0.87702405 0.12297591]
 [0.5409988  0.45900118]
 [0.6385901  0.36140987]
 [0.88937944 0.11062058]
 [0.38916415 0.6108358 ]
 [0.95563793 0.04436211]
 [0.399889   0.600111  ]
 [0.9560812  0.04391873]
 [0.7461668  0.2538332 ]
 [0.71586686 0.2841331 ]
 [0.79814357 0.20185639]
 [0.9473134  0.05268655]
 [0.8972593  0.10274069]
 [0.80456734 0.19543266]
 [0.8456774  0.15432261]
 [0.7520534  0.24794663]
 [0.9311953  0.06880472]
 [0.8815973  0.11840266]
 [0.83954537 0.16045459]
 [0.8961069  0.10389315]
 [0.8624292  0.13757078]
 [0.937597   0.062403  ]
 [0.961722   0.03827791]
 [0.47952425 0.5204758 ]
 [0.46762574 0.5323742 ]
 [0.5556253  0.44437465]
 [0.8098821  0.19011794]


In [8]:
print("Evaluate on test data")
results = model.evaluate(X_test, labels_test)
print("test loss, test acc:", results)

Evaluate on test data
6/6 [==============================] - 0s 653us/step - loss: 0.6642 - accuracy: 0.6816
test loss, test acc: [0.6642029881477356, 0.6815642714500427]


In [9]:
print("Evaluate on train data")
results = model.evaluate(X_train, labels_train)
print("train loss, train acc:", results)

Evaluate on train data
23/23 [==============================] - 0s 981us/step - loss: 0.6639 - accuracy: 0.6643
train loss, train acc: [0.6639474630355835, 0.6643258333206177]


In [10]:
np.savetxt('test_seen.out', X_test, header="{},{}".format(*X_test.shape), delimiter=",")
np.savetxt('labels_test_seen.out', labels_test, header="{},{}".format(*labels_test.shape), delimiter=",")

In [11]:
w1_eval = np.transpose(model.layers[0].get_weights()[0])
w2_eval = np.transpose(model.layers[2].get_weights()[0])
wo_eval = np.transpose(model.layers[4].get_weights()[0])

b1_eval = model.layers[0].get_weights()[1]
b2_eval = model.layers[2].get_weights()[1]
bo_eval = model.layers[4].get_weights()[1]

np.savetxt('w1.out', w1_eval, header="{},{}".format(*w1_eval.shape), delimiter=",")
np.savetxt('w2.out', w2_eval, header="{},{}".format(*w2_eval.shape), delimiter=",")
np.savetxt('wo.out', wo_eval, header="{},{}".format(*wo_eval.shape), delimiter=",")
np.savetxt('b1.out', b1_eval, header="{},{}".format(b1_eval.shape[0], 1), delimiter=",")
np.savetxt('b2.out', b2_eval, header="{},{}".format(b2_eval.shape[0], 1), delimiter=",")
np.savetxt('bo.out', bo_eval, header="{},{}".format(bo_eval.shape[0], 1), delimiter=",")

In [12]:
X_test.shape

(179, 9)

In [13]:
#labels_test

In [15]:
import onnx

In [16]:
import onnxmltools

In [17]:
onnx_model = onnxmltools.convert_keras(model)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 15 -> 9
The maximum opset needed by this model is only 9.


In [18]:
onnxmltools.utils.save_model(onnx_model, 'model.onnx')